# Apply PCM to CMCC temperature data run

Apply PCM method to CMCC temperature data run (example for a 3D gridded product other than ISAS). Notebook developped in local

In [1]:
import xarray as xr
import numpy as np

import pyxpcm
from pyxpcm.models import pcm

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import seaborn as sns

from Plotter import Plotter #new class for plots in pyxpcm

from PIL import Image

## Get data
***

In [2]:
ds_path = "/home/ref-oc-intranet/cmcc_mfseas4/best_estimate/2020/CMCC_MFSEAS-4_20200611_TEMP.nc"
ds_path

'/home/ref-oc-intranet/cmcc_mfseas4/best_estimate/2020/CMCC_MFSEAS-4_20200611_TEMP.nc'

In [ ]:
ds = xr.open_dataset(ds_path)
ds

In [ ]:
ds['depth'].values = -np.abs(ds['depth'].values)
ds
#ds['depth'] = -np.abs(ds['depth'].values) # Ensure depth axis is negative

In [ ]:
coords_list = list(ds.coords.keys())

coords_dict = {}
for c in coords_list:
    axis_at = ds[c].attrs.get('axis')
    print(axis_at)
    if axis_at == 'Y':
        #creates dictionary latitude: c
        coords_dict.update({'latitude': c})
    if axis_at == 'X':
        #creates dictionary longitude: c
        coords_dict.update({'longitude': c})
    if axis_at == 'T':
        #creates dictionay time: c
        coords_dict.update({'time': c})

#check number of variables in dict to assign data_type
if 'latitude' in coords_dict and 'longitude' in coords_dict:
    data_type = 'gridded'
else:
    data_type = 'profiles'
    
coords_dict

In [ ]:
ds

## Applying PCM
***

### Create PCM

Choise of z (very important to make a good choise) 

In [ ]:
# z = np.arange(0.,-1000,-10.)
# z = np.linspace(min(ds.depth.values),max(ds.depth.values), num = ds.depth.size)
z = ds['depth'][0:20]
pcm_features = {'temperature': z}
#z

instantiate the PCM

In [ ]:
m = pcm(K=8, features=pcm_features)
m

### Fit Model

In [ ]:
features_in_ds = {'temperature': 'thetao'}
features_zdim='depth'
m.fit(ds, features=features_in_ds, dim=features_zdim)
m

### Classify data

In [ ]:
m.predict(ds, features=features_in_ds, dim=features_zdim, inplace=True);
ds

In [ ]:
m.predict_proba(ds, features=features_in_ds, dim=features_zdim, inplace=True);
#ds

In [ ]:
ds = ds.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of='thetao', outname='thetao' + '_Q', keep_attrs=True, inplace=True)
# TODO: generic name for the variable
#ds

## Visualisation
***

In [ ]:
P = Plotter(ds.squeeze(), m, data_type = 'gridded')

#### 1. Vertical structure of classes

In [ ]:
P.vertical_structure(q_variable = 'thetao_Q')

#### 2. Spatial distribution of classes

In [ ]:
#change extent (calculate extent max in plotter class?)

extent = np.array([min(ds.lon), max(ds.lon), min(ds.lat), max(ds.lat)]) + np.array([-0.1,+0.1,-0.1,+0.1])
P.spatial_distribution(proj=ccrs.PlateCarree(), 
                       extent=extent, 
                       co = {'latitude':'lat', 'longitude':'lon'})

#### 3. Posteriors

In [ ]:
P.plot_posteriors(proj=ccrs.PlateCarree(), extent=extent, 
                       co = {'latitude':'lat', 'longitude':'lon'})

#### 4. Temporal representation 

<font color=red>__TODO__</font> : Temporal representation in class plotter for 3D gridded data